# **Descarga de librerias necesarias**

In [16]:
!pip install scikit-optimize
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from keras.layers import LSTM, Dense
from keras.constraints import NonNeg
from tensorflow import keras
import os


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.9 MB/s eta 0:00:00


# **Loading Data**

# **Data completa**

In [15]:
import pandas as pd
df = pd.read_csv("Datos_imputados_data_2.csv", parse_dates=["Fecha"]) # Agrega el path de la data
del df["Unnamed: 0"]
df.index = df["Fecha"]

fecha_inicio = pd.to_datetime('2020-04-01')  # Inicio corte
fecha_fin = pd.to_datetime('2021-10-01')  # Fin corte

filtro_fechas = (df.index >= fecha_inicio) & (df.index <= fecha_fin)
df.loc[~filtro_fechas].to_csv("Datos Filtrados.csv")
df["Tasa de desocupación "].loc[~filtro_fechas].to_csv("Tasa Filtradas.csv")

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(["Tasa de desocupación ", "Fecha"], axis=1))
# Variable de entrada
X = scaled_data
# Variable de salida
Y = df["Tasa de desocupación "].values

#per_train = float(input("Ingresar % de datos de entrenamiento: (EJ: 0.8)"))
#X_train, X_test, y_train, y_test = train_test_split(X, Y,
#                                                    train_size=1,
#                                                    random_state= 10
#                                                   )

NameError: ignored

In [ ]:
df.columns

Index(['Índice de Producción Industrial1/Ref/P',
       'Imacec empalmado, desestacionalizado (índice 2018=100)',
       'Índice de Actividad del Comercio al por Menor empalmado1/Ref/P/R',
       'Superficie total1/P', 'M1', 'M2', 'M3', '1.Tasas BCP 10 años',
       'Captaciones 30 a 89 Dias Reaj. Nominal.',
       'Colocaciones 90 a 365 Dias Reaj. U.F.', 'Índice IPC', 'Índice IPP',
       '1.IPSA  (índice enero 2003=1000)', 'Índice Real de Remuneraciones',
       '1.Índice de Avisos Laborales de Internet', 'Price_Copper',
       'Price_oil_WTI', 'Tasa de desocupación ', 'Fecha'],
      dtype='object')

# **Data filtrada**

Columnas a Usar
```
'Índice de Producción Industrial1/Ref/P',
'Imacec empalmado, desestacionalizado (índice 2018=100)',
'Índice de Actividad del Comercio al por Menor empalmado1/Ref/P/R',
'Superficie total1/P', 
'M1', 
'M2', 
'M3', 
'1.Tasas BCP 10 años',
'Captaciones 30 a 89 Dias Reaj. Nominal.',
'Colocaciones 90 a 365 Dias Reaj. U.F.', 
'Índice IPC', 
'Índice IPP',
'1.IPSA  (índice enero 2003=1000)', 
'Índice Real de Remuneraciones',
'1.Índice de Avisos Laborales de Internet', 
'Price_Copper',
'Price_oil_WTI', 
'Tasa de desocupación ',
```



In [ ]:

if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
  
df = pd.read_csv("Datos_imputados_data_2.csv").iloc[:-3] # Agregar path de la data
del df["Unnamed: 0"]
df = df[df.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]
#df.to_csv("drive/MyDrive/output_benchmark/Datos Imputados y Filtrados.csv")
df.index = pd.to_datetime(df["Fecha"])
'''
fecha_inicio = pd.to_datetime('2020-04-01')#Inicio corte
fecha_fin = pd.to_datetime('2021-10-01')#Fin corte

filtro_fechas = (df.index >= fecha_inicio) & (df.index <= fecha_fin)
df = df.drop(df.loc[filtro_fechas, 'Tasa de desocupación '].index,axis=0)
'''
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(["Tasa de desocupación ", "Fecha"], axis=1))
# Variable de entrada
X = scaled_data
# Variable de salida
Y = df["Tasa de desocupación "].values

In [ ]:
len(Y)

155

# **RandomForestRegressor**

- Utiliza una busqueda bayesiana para los hyperparametros 
  - Parametros a buscar:
    1.   random_state
    2.   max_depth
    3. criterion

*   cv = Cross-validation
*   iter = Iteraciones





In [ ]:
opt = BayesSearchCV(
    RandomForestRegressor(),
    {
        'random_state': Integer(0, 5, prior='uniform'),
        'max_depth': Integer(1, 5, prior='uniform'),
        'criterion': Categorical(["squared_error", "absolute_error", "friedman_mse", "poisson"]),

    },
    n_iter=40,
    cv=10
)

# executes bayesian optimization
_ = opt.fit(X, Y) 
# aqui termina la busqueda

# model can be saved, used for predictions or scoring

#y_pred = (opt.predict(X_test))

#ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha RFR"])
#testYdf = pd.DataFrame(data = y_test)


#pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

#r2 = r2_score(
#    y_true=y_test,
#    y_pred = y_pred)
#print(f"El valor de R2 es:{r2}")
#mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
#print(f"El valor de MSE es:{mse}")
#rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
#print(f"El valor de RMSE es:{rmse}")
##Guardando Modelo
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
filename = 'drive/MyDrive/output_benchmark/finalized_model_RandomForestRegressor.sav'
pickle.dump(_, open(filename, 'wb'))

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist

# **SVR**
- Utiliza una busqueda bayesiana para los hyperparametros
  - Parametros a buscar:
    1.   C
    2.   epsilon
    3. kernel

*   cv = Cross-validation
*   iter = Iteraciones



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
import pickle


results = pd.DataFrame(columns= ["Iteracion","C","epsilon","kernel","R2","Path"])
numero_iteracion = 0
iteraciones = []
C_ = []
epsilon_ = []
kernel_ = []
r2_ = []
path_model_= []


# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    SVR(),
    {
        'C': Real(.01, 100, prior='log-uniform'),
        'epsilon': Real(.0001, 2, prior='log-uniform'),
        'kernel': Categorical(['linear', 'poly', 'rbf']),

    },
    n_iter=100,
    cv=10
)

# executes bayesian optimization
_ = opt.fit(X, Y)

# model can be saved, used for predictions or scoring
#y_pred = opt.predict(X_test)

#newdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha SVR"])
#testYdf = pd.DataFrame(data = y_test)


#pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

#r2 = r2_score(
#    y_true=y_test,
#    y_pred = y_pred)
#
#
#r2 = r2_score(
#    y_true=y_test,
#    y_pred = y_pred)
#print(f"El valor de R2 es:{r2}")
#mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
#print(f"El valor de MSE es:{mse}")
#rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
#print(f"El valor de RMSE es:{rmse}")
#Guardando Modelo
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
filename = 'drive/MyDrive/output_benchmark/finalized_model_SVR.sav'
pickle.dump(_, open(filename, 'wb'))

# **Multi-layer Perceptron regressor.**
- Utiliza una busqueda bayesiana para los hyperparametros
  - Parametros a buscar:
    1.   alpha
    2.   epsilon
    3. activation
    4. solver

*   cv = Cross-validation
*   iter = Iteraciones



In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
import pickle


results = pd.DataFrame(columns= ["Iteracion","C","epsilon","kernel","R2","Path"])
numero_iteracion = 0
iteraciones = []
C_ = []
epsilon_ = []
kernel_ = []
r2_ = []
path_model_= []


# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    MLPRegressor(),
    {
        'alpha': Real(.0001, 1, prior='log-uniform'),
        'epsilon': Real(.0001, 2, prior='log-uniform'),
        'activation': Categorical(['identity', 'logistic', 'tanh', 'relu']),
        'solver': Categorical(['lbfgs', 'sgd', 'adam'])

    },
    n_iter=100,
    cv=5
)

# executes bayesian optimization
_ = opt.fit(X, Y)
'''
# model can be saved, used for predictions or scoring
y_pred = opt.predict(X_test)

ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha MLPRegressor"])
testYdf = pd.DataFrame(data = y_test)


pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)


r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)
print(f"El valor de R2 es:{r2}")
mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
print(f"El valor de MSE es:{mse}")
rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
print(f"El valor de RMSE es:{rmse}")'''
#Guardando Modelo
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
filename = 'drive/MyDrive/output_benchmark/finalized_model_MLPRegressor.sav'
pickle.dump(_, open(filename, 'wb'))

# **Lasso Regression**

- Utiliza una busqueda bayesiana para los hyperparametros
  - Parametros a buscar:
    1.   alpha

*   cv = Cross-validation
*   iter = Iteraciones




In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
import pickle


results = pd.DataFrame(columns= ["Iteracion","C","epsilon","kernel","R2","Path"])
numero_iteracion = 0
iteraciones = []
C_ = []
epsilon_ = []
kernel_ = []
r2_ = []
path_model_= []


# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    Lasso(),
    {
        'alpha': Real(.0001, 1, prior='log-uniform')

    },
    n_iter=100,
    cv=10
)

# executes bayesian optimization
_ = opt.fit(X_train, y_train)
'''
# model can be saved, used for predictions or scoring
y_pred = opt.predict(X_test)

ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha Lasso"])
testYdf = pd.DataFrame(data = y_test)


pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)


r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)
print(f"El valor de R2 es:{r2}")
mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
print(f"El valor de MSE es:{mse}")
rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
print(f"El valor de RMSE es:{rmse}")'''
#Guardando Modelo
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
  
filename = 'drive/MyDrive/output_benchmark/finalized_model_Lasso.sav'
pickle.dump(_, open(filename, 'wb'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.713e-01, tolerance: 2.431e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.165e+00, tolerance: 2.388e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.133e-01, tolerance: 2.649e

# **DecisionTreeRegressor**

- Utiliza una busqueda bayesiana para los hyperparametros
  - Parametros a buscar:
   1.   max_depth



*   cv = Cross-validation
*   iter = Iteraciones




In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score 
import pickle


results = pd.DataFrame(columns= ["Iteracion","C","epsilon","kernel","R2","Path"])
numero_iteracion = 0
iteraciones = []
C_ = []
epsilon_ = []
kernel_ = []
r2_ = []
path_model_= []


# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    DecisionTreeRegressor(),
    {
        'max_depth': Integer(1, 10, prior='log-uniform')

    },
    n_iter=10,
    cv=10
)

# executes bayesian optimization
_ = opt.fit(X, Y)
'''
# model can be saved, used for predictions or scoring
y_pred = opt.predict(X_test)

ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha SVR"])
testYdf = pd.DataFrame(data = y_test)


pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)


r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)
print(f"El valor de R2 es:{r2}")
mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
print(f"El valor de MSE es:{mse}")
rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
print(f"El valor de RMSE es:{rmse}")

'''
#Guardando Modelo
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
  
filename = 'drive/MyDrive/output_benchmark/finalized_model_DecisionTreeRegressor.sav'
pickle.dump(_, open(filename, 'wb'))

# **KNeighborsRegressor**

- Utiliza una busqueda bayesiana para los hyperparametros
  - Parametros a buscar:
    1.   n_neighbors
    2. algorithm

*   cv = Cross-validation
*   iter = Iteraciones



In [ ]:

results = pd.DataFrame(columns= ["Iteracion","C","epsilon","kernel","R2","Path"])
numero_iteracion = 0
iteraciones = []
C_ = []
epsilon_ = []
kernel_ = []
r2_ = []
path_model_= []


# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    KNeighborsRegressor(),
    {
        'n_neighbors': Integer(1, 10, prior='log-uniform'),
        'algorithm': Categorical(['auto', 'ball_tree', 'kd_tree','brute'])

    },
    n_iter=40,
    cv=10
)

# executes bayesian optimization
_ = opt.fit(X, Y)
'''
# model can be saved, used for predictions or scoring
y_pred = opt.predict(X_test)

ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha SVR"])
testYdf = pd.DataFrame(data = y_test)


pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)


r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)
print(f"El valor de R2 es:{r2}")
mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
print(f"El valor de MSE es:{mse}")
rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
print(f"El valor de RMSE es:{rmse}")
#Guardando Modelo'''
data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta que se va a generar en el drive
if os.path.isdir(data_folder) == False: # Ask if the folder exist, if return a false create the folder
  os.mkdir(data_folder)
filename = 'drive/MyDrive/output_benchmark/finalized_model_KNN.sav'
pickle.dump(_, open(filename, 'wb'))

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist

# **LSTM MultiVariate**
- Optimización utilizando hypeband, buscar hyperband al final del Colab
- Se hace un reshape de los datos de entrenamiento y testeo
- **Copiar los resultados del hyperband en la creación del modelo**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from keras.layers import LSTM, Dense
from keras.constraints import NonNeg
from tensorflow import keras
# Cargar los datos
#df = pd.read_csv("Datos_imputados.csv")

X_train_ = X_train.reshape((X_train.shape[0], 1, X_train.shape[1])) 
X_test_ = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear el modelo LSTM
model = Sequential()
#Se escribe a mano los resultados del hyperband que se ejecuta al final del colab
model.add(LSTM(68, activation='relu', input_shape=(1, X_train_.shape[2])))
model.add(Dense(units=92,activation='relu',kernel_initializer='HeNormal'))
model.add(Dense(units=164,activation='relu',kernel_initializer='HeNormal'))
model.add(Dense(units=296,activation='relu',kernel_initializer='HeNormal'))
model.add(Dense(1, activation='linear',kernel_initializer='normal')) 
###################
model.compile(optimizer=keras.optimizers.Adam(0.01), loss='mse',)

# Entrenar el modelo
model.fit(X_train_, y_train, epochs=100, batch_size=3, verbose=1)

# Realizar predicciones
x_input = X_test
y_pred = model.predict(x_input)

ynewdf = pd.DataFrame(data = y_pred,columns=["Tasa de desocupación predicha LSTM"])
testYdf = pd.DataFrame(data = y_test)

pd.concat([testYdf.reset_index().drop("index",axis=1),ynewdf],axis=1).plot()

r2 = r2_score(
    y_true=y_test,
    y_pred = y_pred)
print(f"El valor de R2 es:{r2}")
mse = mean_squared_error(y_test,y_pred,squared=True) # True calcula el MSE
print(f"El valor de MSE es:{mse}")
rmse = mean_squared_error(y_test,y_pred,squared=False)# False calcula el RMSE
print(f"El valor de RMSE es:{rmse}")

Hyperband LSTM

---


> Primer paso

Se define la neurona inicial:

```
model.add(LSTM(68, activation='relu', input_shape=(1, X_train_.shape[2])))
```

> Segundo paso

 Para cambiar los parametros de busqueda en el hyperband se modifica en primer lugar: `for i in range(hp.Int('num_of_layers',2,10))` para cambiar el rango de el numero de layers a utilizar en este caso son de 2 a 10.

> Tercer paso

Las Capas Dense donde se modifican el numero de neuronas que cada una puede tener.

```
# model.add(Dense(units=hp.Int('num_of_neurons'+ str(i),min_value=20,max_value=312,step=12),
                                    activation='relu',kernel_initializer='HeNormal'))
```


> Cuarto paso

Modificar los valores del optimizador.
```
# model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate',values=[1e-2, 1e-3, 1e-4])),loss='mse',
                  metrics=['mse'])
```








In [ ]:
!pip install keras-tuner
from keras_tuner import Hyperband



def build_model(hp): # Función de hyperband
    model=Sequential()
    model.add(LSTM(68, activation='relu', input_shape=(1, X_train_.shape[2])))
    for i in range(hp.Int('num_of_layers',2,30)):         
      #providing range for number of neurons in hidden layers
      #Activation Opciones : sigmoid,relu, LeakyRelu , relu entrega los mejores resultados para este caso.
      # Como son numeros positivos relativamente altos, es recomendado utilizar relu.
      # En caso contrario, probar utilizar Sigmoid o Leakyrelu
       model.add(Dense(units=hp.Int('num_of_neurons'+ str(i),min_value=20,max_value=312,step=12),
                                    activation='relu',kernel_initializer='HeNormal'))
    model.add(Dense(1, activation='linear',kernel_initializer='normal'))    #output layer
    #compiling the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning rate',values=[1e-2, 1e-3, 1e-4])),loss='mse',
                  metrics=['mse'])
    return model
#Inicia el Hyperband
X_train_ = X_train.reshape((X_train.shape[0], 1, X_train.shape[1])) 
X_test_ = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))




tuner=Hyperband(build_model, objective='mse',
                    directory='Hyperband Opt4', project_name= 'fixed2')
tuner.search(X_train_,y_train,epochs=1,validation_data=(X_test_,y_test))
#
#Resultados
tuner.results_summary()

# **Cargando Modelos**

Modelos disponibles para testeo se utiliza la misma data de test. 

*   finalized_model_KNN.sav (KNN)
*   finalized_model_DecisionTreeRegressor.sav (DTR)
*   finalized_model_Lasso.sav (Lasso)
*   finalized_model_MLPRegressor.sav (MLP)
*   finalized_model_SVR.sav (SVR)
*   finalized_model_RandomForestRegressor.sav (RFR)


---

**Los modelos se crearan luego de ejecutar por primera ves los codigos.**




In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_SVR.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
SVR = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha SVR"])


La tasa de desocupación para la muestra ingresada es de:[7.93115571 8.2211726  8.07702477]


In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_Lasso.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
lasso = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha LASSO"])

La tasa de desocupación para la muestra ingresada es de:[10.52981326 18.62448785  8.13423605]


In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_RandomForestRegressor.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
RFR = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha RFR"])

La tasa de desocupación para la muestra ingresada es de:[ 7.77583422  7.77695067 10.55810894]


In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_KNN.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
KNN = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha KNN"])

La tasa de desocupación para la muestra ingresada es de:[ 7.81147375 11.57703876  8.12456869]


In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_DecisionTreeRegressor.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
DT = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha DTR"])

La tasa de desocupación para la muestra ingresada es de:[ 7.50377726  7.50377726 10.31592888]


In [ ]:
df2 = pd.read_csv("Datos_imputados_data_2.csv").iloc[-3:] 
del df2["Unnamed: 0"]
df2 = df2[df2.columns[[0,2,3,4,7,9,11,13,14,15,16,17,18,20,21,22,24,25,26]]]

data_folder = "drive/MyDrive/output_benchmark/" #nombre de la carpeta contenedora
scaled_data_test = scaler.fit_transform(df2.drop(["Tasa de desocupación ", "Fecha"], axis=1))
pickled_model = pickle.load(open(f'{data_folder}finalized_model_MLPRegressor.sav', 'rb')) # AQUI SE CAMBIA EL MODELO A UTILIZAR POR UNOS DE LOS QUE SE NOMBRAN ARRIBA
tasa_desocupacion_predicha = pickled_model.predict(scaled_data_test) #Predict
print(f"La tasa de desocupación para la muestra ingresada es de:{tasa_desocupacion_predicha}")
MLP = pd.DataFrame(data = tasa_desocupacion_predicha,columns=["Tasa de desocupación predicha MLP"])

La tasa de desocupación para la muestra ingresada es de:[ 7.68645108 18.04309271  9.21328662]


In [ ]:

final = pd.concat([df2["Tasa de desocupación "].reset_index().drop("index",axis=1),KNN],axis=1)
final = pd.concat([final,DT],axis=1)
final = pd.concat([final,SVR],axis=1)

final = pd.concat([final,RFR],axis=1)

final = pd.concat([final,lasso],axis=1)
final = pd.concat([final,MLP],axis=1)
final.index = df2["Fecha"]
final.to_csv("Resultados_regresión.csv")

In [ ]:
final
for x in final:
  if x != "Tasa de desocupación ":
    final[x] = final["Tasa de desocupación "] - final[x]

In [ ]:
final

,Tasa de desocupación,Tasa de desocupación predicha KNN,Tasa de desocupación predicha DTR,Tasa de desocupación predicha SVR,Tasa de desocupación predicha RFR,Tasa de desocupación predicha LASSO,Tasa de desocupación predicha MLP
Fecha,,,,,,,
2023-02-01,8.367641,0.556167,0.863863,0.436485,0.591806,-2.162173,0.681190
2023-03-01,8.808479,-2.768560,1.304702,0.587306,1.031528,-9.816009,-9.234614
2023-04-01,8.659709,0.535140,-1.656220,0.582684,-1.898400,0.525473,-0.553578
